In [1]:
import os
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )

    

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description="""
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    

class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()
    
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool()
    ],
)


prompt = "Give me financial information on CRISPR Therapeutics A's stock, considering its financials, income statements and stock performance and help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...


Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Invalid API key"}')



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Pfizer stock market symbol'}`




d:\project_python\fullstack-gpt\venv\Lib\site-packages\curl_cffi\aio.py:38: RuntimeWarning: 
    Proactor event loop does not implement add_reader family of methods required.
    Registering an additional selector thread for add_reader support.
    To avoid this warning use:
        asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
    
  warnings.warn(PROACTOR_WARNING, RuntimeWarning)
Authentication failed for https://api.smith.langchain.com/runs/f028d802-d500-4266-bc86-f7cf89b73a40. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/f028d802-d500-4266-bc86-f7cf89b73a40', '{"detail":"Invalid API key"}')


Ticker Symbol PFE Full Company Profile Financial Performance In 2022, Pfizer's revenue was $100.33 billion, an increase of 23.43% compared to the previous year's $81.29 billion. Earnings were $31.37 billion, an increase of 42.74%. Financial Statements Analyst Forecast According to 15 analysts, the average rating for PFE stock is "Buy." January 11, 2024 · 3 min read Post-pandemic, the global bio-pharmaceutical sector, often overlooked, presents undervalued opportunities, notably in blue-chip stocks like Pfizer (NYSE: PFE ).... Pfizer is expected to post earnings of $0.53 per share for the current quarter, representing a year-over-year change of -56.9%. Over the last 30 days, the Zacks Consensus Estimate has changed -12. ... Pfizer MarketRank™ Stock Analysis Analyst Rating Hold 2.33 Rating Score Upside/ Downside 33.7% Upside $36.93 Price Target Price vs Fair Value Sustainability Trailing Returns Financials Valuation Operating Performance Dividends Ownership Executive $27.62 +0.11 (0.40%)

{'input': "Give me financial information on Pfizer's stock, considering its financials, income statements and stock performance and help me analyze if it's a potential good investment.",
 'output': "Here's the financial information for Pfizer Inc. (PFE):\n\n1. Company Overview:\n   - Name: Pfizer Inc\n   - Description: Pfizer Inc. is an American multinational pharmaceutical and biotechnology corporation headquartered in New York City. The company develops and produces medicines and vaccines for various medical fields.\n   - Market Capitalization: $156.24 billion\n   - Dividend Per Share: $1.64\n   - Dividend Yield: 0.0607\n   - Revenue (TTM): $58.50 billion\n   - Gross Profit (TTM): $66.23 billion\n   - Earnings Per Share (EPS): $0.37\n   - Price/Earnings Ratio (P/E): 74.78\n   - Forward P/E Ratio: 12.47\n   - Analyst Target Price: $32.14\n   - Analyst Ratings: 3 Strong Buy, 7 Buy, 15 Hold, 0 Sell, 0 Strong Sell\n\n2. Income Statement (2022):\n   - Total Revenue: $100.33 billion\n   - 

d:\project_python\fullstack-gpt\venv\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
